# Scénario 3 : Préparation d'un Jeu de Données Archéologiques pour Dépôt FAIR (type NAKALA)

**Chercheuse (fictive) :** Sophie Bernard, archéologue.

**Phase du projet :** Fin de recherche. Elle a analysé sa collection d'artefacts et souhaite déposer son jeu de données (descriptions tabulaires, mesures, photographies, rapport de synthèse) de manière FAIR (Facile à trouver, Accessible, Interopérable, Réutilisable) et le valoriser.

**Objectif de cet accompagnement (simulé via ce notebook) :**
1. Conseiller sur la structuration d'un jeu de données hétérogènes pour un dépôt.
2. Aider à la création d'un fichier de métadonnées descriptives standardisé pour le jeu de données global (Dublin Core).
3. Préparer un "paquet de dépôt" organisant les fichiers et les métadonnées, en vue d'un dépôt sur un entrepôt de données comme NAKALA (proposé par Huma-Num).
4. Fournir des conseils sur le choix d'une licence de réutilisation.

---
**Méthodologie :**
* Présentation des principes FAIR et de la structuration d'un paquet de dépôt.
* Utilisation de Python pour organiser les fichiers et générer les fichiers de métadonnées, de documentation et de licence.
* Simulation des étapes de réflexion avant un dépôt sur un entrepôt comme NAKALA.

**Note importante :** cette démonstration tend à compliquer des processus qui peuvent être réalisés de manière beaucoup plus simples, elle est surtout à voir comme une démonstration technique plutôt qu'un réel workflow prêt à l'emploi. 

## 0. Importation des bibliothèques nécessaires

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
from xml.dom import minidom
import zipfile
import datetime
import json 

## 1. Définition du Jeu de Données d'Exemple et des Paramètres du Dépôt

Pour cette démonstration, nous allons travailler avec un jeu de données archéologiques fictif que Sophie Bernard souhaite déposer.
Ce jeu de données source est supposé exister dans un dossier `source_data_archeo/` à côté de ce notebook.

**Contenu supposé de `source_data_archeo/` :**
* `inventaire_artefacts.csv` : Inventaire tabulaire des artefacts. (Inventaire factice généré avec Gemini 2.5 le 21 mai 2025)
* `images/` (dossier) : Contenant des fichiers `.jpg` (ex: `VF_2023_CER_001.jpg`, `VF_2024_MET_002.jpg`). (Ce sont des fichiers vides, utilisés uniquement pour l'illustration du processus)
* `rapport_synthese_VillaFloriana_2024.md` : Rapport de synthèse des fouilles. (Rapport factice généré avec Gemini 2.5 le 21 mai 2025)

In [ ]:
SOURCE_DATA_DIR = 'source_data_archeo/'
OUTPUT_DIR_BASE = 'output_depot/' # Dossier où sera créé le paquet de dépôt
DEPOSIT_PACKAGE_NAME = 'depot_archeo_VillaFloriana_2024' # Nom du dossier principal du paquet

# Chemin complet vers le paquet de dépôt
deposit_path = os.path.join(OUTPUT_DIR_BASE, DEPOSIT_PACKAGE_NAME)

# Création des dossiers de sortie s'ils n'existent pas
if not os.path.exists(OUTPUT_DIR_BASE):
    os.makedirs(OUTPUT_DIR_BASE)

# Si un ancien paquet existe, on le supprime pour cette démo (pour repartir de zéro)
if os.path.exists(deposit_path):
    shutil.rmtree(deposit_path)
os.makedirs(deposit_path)

print(f"Le paquet de dépôt sera créé dans : {os.path.abspath(deposit_path)}")
print(f"Vérifiez que le dossier source '{os.path.abspath(SOURCE_DATA_DIR)}' existe et contient les fichiers d'exemple.")

## 2. Principes Directeurs : Structuration et FAIR Data

Avant de créer le paquet, rappelons quelques principes importants pour le partage des données de recherche :

### 2.1. Organisation des Fichiers
Un jeu de données bien organisé est plus facile à comprendre et à réutiliser. Nous allons adopter une structure de dossiers simple et logique :
* `data/` : pour les données tabulaires (CSV, etc.).
* `images/` : pour les fichiers images.
* `documentation/` : pour les rapports, notes méthodologiques, et le `README_dataset.md`.
Les métadonnées (XML/JSON), la licence, et un README général du dépôt peuvent être à la racine du paquet.

### 2.2. Formats Ouverts et Pérennes
Privilégier des formats non-propriétaires, bien documentés, et susceptibles d'être lisibles sur le long terme :
* **Données tabulaires :** CSV (avec encodage UTF-8), TSV.
* **Texte :** TXT (UTF-8), Markdown (.md), PDF/A (pour l'archivage de documents mis en page).
* **Images :** TIFF (archivage), PNG (sans perte), JPEG (avec perte, mais courant).

### 2.3. Principes FAIR
L'objectif est de rendre les données **F**aciles à trouver, **A**ccessibles, **I**nteropérables, et **R**éutilisables.
* **Facile à trouver (Findable):** Métadonnées riches, identifiant pérenne (DOI fourni par l'entrepôt comme NAKALA).
* **Accessible :** Données déposées dans un entrepôt ouvert ou avec des conditions d'accès claires.
* **Interopérable :** Utilisation de formats standards, de vocabulaires contrôlés pour les métadonnées.
* **Réutilisable (Reusable) :** Licence claire, documentation complète (contexte, méthodologie, structure des données).

## 3. Préparation du Paquet de Dépôt : Copie et Organisation des Fichiers

In [ ]:
# Définition des sous-dossiers cibles dans le paquet de dépôt
target_data_dir = os.path.join(deposit_path, 'data')
target_images_dir = os.path.join(deposit_path, 'images')
target_docs_dir = os.path.join(deposit_path, 'documentation')

# Création de l'arborescence
os.makedirs(target_data_dir, exist_ok=True)
os.makedirs(target_images_dir, exist_ok=True)
os.makedirs(target_docs_dir, exist_ok=True)

print(f"Arborescence créée dans '{deposit_path}'")

# Fonction pour copier les fichiers sources vers le paquet de dépôt
def copy_source_files(source_dir, deposit_target_path):
    copied_files_count = 0
    if not os.path.exists(source_dir):
        print(f"ERREUR: Le dossier source '{source_dir}' n'existe pas. Veuillez le créer et y placer les fichiers d'exemple.")
        return copied_files_count
    
    # Copie de inventaire_artefacts.csv
    source_csv = os.path.join(source_dir, 'inventaire_artefacts.csv')
    if os.path.exists(source_csv):
        shutil.copy2(source_csv, os.path.join(deposit_target_path, 'data', 'inventaire_artefacts.csv'))
        print(f"- '{source_csv}' copié vers '{os.path.join(deposit_target_path, 'data')}'")
        copied_files_count +=1
    else:
        print(f"- AVERTISSEMENT: '{source_csv}' non trouvé.")

    # Copie du rapport
    # Chercher un rapport .md ou .txt
    report_file_name = None
    if os.path.exists(os.path.join(source_dir, 'rapport_synthese_VillaFloriana_2024.md')):
        report_file_name = 'rapport_synthese_VillaFloriana_2024.md'
    elif os.path.exists(os.path.join(source_dir, 'rapport_synthese_VillaFloriana_2024.txt')):
        report_file_name = 'rapport_synthese_VillaFloriana_2024.txt'
        
    if report_file_name:
        source_report = os.path.join(source_dir, report_file_name)
        shutil.copy2(source_report, os.path.join(deposit_target_path, 'documentation', report_file_name))
        print(f"- '{source_report}' copié vers '{os.path.join(deposit_target_path, 'documentation')}'")
        copied_files_count +=1
    else:
        print(f"- AVERTISSEMENT: Rapport de synthèse (.md ou .txt) non trouvé dans '{source_dir}'.")

    # Copie des images
    source_images_dir = os.path.join(source_dir, 'images')
    target_img_dir_in_deposit = os.path.join(deposit_target_path, 'images')
    if os.path.exists(source_images_dir):
        for img_filename in os.listdir(source_images_dir):
            if img_filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                shutil.copy2(os.path.join(source_images_dir, img_filename), os.path.join(target_img_dir_in_deposit, img_filename))
                print(f"  - Image '{img_filename}' copiée vers '{target_img_dir_in_deposit}'")
                copied_files_count +=1
    else:
        print(f"- AVERTISSEMENT: Dossier d'images source '{source_images_dir}' non trouvé.")
    return copied_files_count

print("\nCopie des fichiers sources vers le paquet de dépôt...")
nb_copied = copy_source_files(SOURCE_DATA_DIR, deposit_path)
if nb_copied == 0:
    print("\nAUCUN FICHIER SOURCE N'A ÉTÉ COPIÉ. VÉRIFIEZ VOTRE DOSSIER 'source_data_archeo/' ET SON CONTENU.")

## 4. Création du Fichier de Métadonnées Descriptives (Dublin Core)

Nous allons générer un fichier `metadata_dublincore.xml` pour décrire le jeu de données dans son ensemble. NAKALA s'appuie sur Dublin Core et DataCite.
Pour cette démo, nous utilisons des valeurs prédéfinies.

In [ ]:
dataset_metadata = {
    "title": "Jeu de données archéologiques du site de la Villa Floriana (Bourgogne) - Campagne 2024",
    "creator": "Bernard, Sophie (Dr.)",
    "contributor": "Équipe Archéologique Inrap Grand Est (fictif)",
    "subject": [
        "Archéologie", 
        "Gallo-Romain", 
        "Bourgogne", 
        "Villa Floriana", 
        "Artefacts", 
        "Céramique", 
        "Numismatique"
    ],
    "description": "Ce jeu de données comprend l'inventaire des artefacts (CSV), une sélection de photographies (JPG), et le rapport de synthèse (MD) issus des fouilles préventives menées en 2024 sur le site gallo-romain de la Villa Floriana en Bourgogne. Le site a livré des vestiges d'occupation s'étalant principalement du Ier au IVe siècle ap. J.-C.",
    "publisher": "Université de Bourgogne (fictif) / Laboratoire ARTeHIS UMR 6298 (fictif)",
    "date": datetime.date.today().isoformat(), # Date de création du paquet de dépôt
    "type": "Dataset", # Ou "Collection" si on considère l'ensemble des fichiers
    "format": ["text/csv", "image/jpeg", "text/markdown"],
    "identifier": "(sera assigné par NAKALA lors du dépôt, ex: DOI)",
    "language": "fra", # Français
    "rights": "Creative Commons Attribution 4.0 International (CC BY 4.0)",
    "license_url": "https://creativecommons.org/licenses/by/4.0/",
    "coverage_spatial": "Villa Floriana, Bourgogne, France (coordonnées fictives : 47.3220 N, 5.0415 E)",
    "coverage_temporal": "Ier siècle ap. J.-C. - IVe siècle ap. J.-C."
}

def create_dublin_core_xml(metadata_dict, output_path):
    # Racine de l'élément avec namespace Dublin Core et RDF
    root = ET.Element("metadata", {
        "xmlns:dc": "http://purl.org/dc/elements/1.1/",
        "xmlns:dcterms": "http://purl.org/dc/terms/",
        "xmlns:rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#" 
    })

    simple_dc_map = {
        "title": "dc:title",
        "creator": "dc:creator",
        "publisher": "dc:publisher",
        "type": "dc:type",
        "language": "dc:language",
        "description": "dcterms:abstract", 
        "contributor": "dc:contributor",
        "identifier": "dc:identifier",
        "rights": "dcterms:accessRights" 
    }

    for key, dc_element_name in simple_dc_map.items():
        if key in metadata_dict and metadata_dict[key]:
            el = ET.SubElement(root, dc_element_name)
            el.text = str(metadata_dict[key])

    # Champs potentiellement multiples (subject, format)
    if "subject" in metadata_dict:
        for subj in metadata_dict["subject"]:
            el = ET.SubElement(root, "dc:subject")
            el.text = subj
    if "format" in metadata_dict:
        for fmt in metadata_dict["format"]:
            el = ET.SubElement(root, "dc:format")
            el.text = fmt

    # Dates
    if "date" in metadata_dict: # date de publication du dataset
        el = ET.SubElement(root, "dc:date") 
        el.text = metadata_dict["date"]
    
    # Coverage
    if "coverage_spatial" in metadata_dict:
        el = ET.SubElement(root, "dcterms:spatial")
        el.text = metadata_dict["coverage_spatial"]
    if "coverage_temporal" in metadata_dict:
        el = ET.SubElement(root, "dcterms:temporal")
        el.text = metadata_dict["coverage_temporal"]
    
    # Licence URL (dcterms:license)
    if "license_url" in metadata_dict:
        el = ET.SubElement(root, "dcterms:license")
        el.set("rdf:resource", metadata_dict["license_url"]) # Attribut pour URL
        el.text = metadata_dict["rights"] # Le nom de la licence en texte


    # "Pretty print" XML
    xml_str = ET.tostring(root, encoding='utf-8', method='xml')
    dom = minidom.parseString(xml_str)
    pretty_xml_str = dom.toprettyxml(indent="  ", encoding="utf-8")

    with open(os.path.join(output_path, "metadata_dublincore.xml"), "wb") as f:
        f.write(pretty_xml_str)
    print(f"- Fichier 'metadata_dublincore.xml' créé dans '{output_path}'.")

create_dublin_core_xml(dataset_metadata, deposit_path)

# Optionnel: Créer un JSON simple aussi
json_metadata_path = os.path.join(deposit_path, "metadata_simple.json")
with open(json_metadata_path, 'w', encoding='utf-8') as f_json:
    json.dump(dataset_metadata, f_json, ensure_ascii=False, indent=4)
print(f"- Fichier 'metadata_simple.json' créé dans '{deposit_path}'.")

## 5. Création des Fichiers Complémentaires : `README_dataset.md` et `LICENSE.txt`

### 5.1. Création du `README_dataset.md`
Ce fichier est crucial pour expliquer le contenu et le contexte du jeu de données aux futurs utilisateurs.

In [ ]:
readme_content = f"""
# Jeu de Données : {dataset_metadata['title']}

**Date de création du paquet :** {dataset_metadata['date']}

## Auteur(s) et Contact

- **Créateur principal :** {dataset_metadata['creator']}
- **Contributeurs :** {dataset_metadata.get('contributor', 'N/A')}
- **Institution de publication :** {dataset_metadata['publisher']}
- **Contact :** [email_chercheur@exemple.com]

## Description du Jeu de Données

{dataset_metadata['description']}

**Mots-clés :** {', '.join(dataset_metadata['subject'])}

**Couverture géographique :** {dataset_metadata['coverage_spatial']}
**Couverture temporelle des données :** {dataset_metadata['coverage_temporal']}

## Méthodologie de Collecte/Production

Les données ont été collectées lors de fouilles archéologiques préventives sur le site de la Villa Floriana (Bourgogne) durant la campagne 2024. 
L'inventaire des artefacts a été réalisé selon les standards du [Nom de l'institution ou du projet]. 
Les photographies ont été prises en [Conditions de prise de vue].
Le rapport de synthèse fournit un aperçu des découvertes.
[Ajouter plus de détails sur la méthodologie spécifique si nécessaire]

## Structure du Jeu de Données et Formats des Fichiers

Le jeu de données est structuré comme suit :

- `data/inventaire_artefacts.csv` : Fichier CSV (séparateur virgule, encodage UTF-8) contenant l'inventaire détaillé des artefacts. 
  Colonnes : `id_artefact`, `type_objet`, `materiau`, `periode_estimee`, `contexte_decouverte`, `description_courte`, `dim_L_cm`, `dim_l_cm`, `dim_h_cm_ou_epaisseur`, `poids_g`, `nom_fichier_photo`.
- `images/` : Dossier contenant les photographies des artefacts sélectionnés (format JPEG).
- `documentation/rapport_synthese_VillaFloriana_2024.md` : Rapport de synthèse des fouilles (format Markdown).
- `metadata_dublincore.xml` : Fichier de métadonnées descriptives du jeu de données au format Dublin Core XML.
- `LICENSE.txt` : Fichier texte décrivant les conditions de réutilisation du jeu de données.
- `README_dataset.md` (ce fichier) : Informations générales sur le jeu de données.

## Conditions de Réutilisation et Citation

Ce jeu de données est diffusé sous la licence **{dataset_metadata['rights']}**. 
Voir le fichier `LICENSE.txt` pour les détails complets de la licence. 
URL de la licence : {dataset_metadata['license_url']}

**Pour citer ce jeu de données :**
Bernard, Sophie (Dr.). ({dataset_metadata['date']}). *{dataset_metadata['title']}*. [{dataset_metadata['type']}]. {dataset_metadata['publisher']}. 
Identifiant : [DOI qui sera assigné par NAKALA lors du dépôt, ex: 10.34847/xxxx]
[Adapter cette suggestion de citation aux normes de la discipline ou aux recommandations de l'entrepôt]

## Informations Complémentaires

[Ajouter ici toute information pertinente non couverte ailleurs, ex: logiciels spécifiques nécessaires pour ouvrir certains fichiers, projets de recherche liés, publications associées, etc.]
"""

readme_dataset_path = os.path.join(deposit_path, 'README_dataset.md') # Placé à la racine pour visibilité
with open(readme_dataset_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)
print(f"- Fichier 'README_dataset.md' créé dans '{deposit_path}'.")

### 5.2. Création du `LICENSE.txt`
Nous choisissons une licence Creative Commons CC BY 4.0, qui est une bonne pratique pour la science ouverte.

In [ ]:
license_content = f"""
Titre du Jeu de Données : {dataset_metadata['title']}
Auteur(s) : {dataset_metadata['creator']}
Date de publication : {dataset_metadata['date']}

Ce jeu de données est mis à disposition sous la licence Creative Commons Attribution 4.0 International (CC BY 4.0).

Vous êtes autorisé à :
- Partager — copier et redistribuer le matériel par tous moyens et sous tous formats.
- Adapter — remixer, transformer et créer à partir du matériel pour toute utilisation, y compris commerciale.

Sous les conditions suivantes :
- Attribution — Vous devez créditer l'Œuvre, intégrer un lien vers la licence et indiquer si des modifications ont été effectuées à l'Oeuvre. Vous devez indiquer ces informations par tous les moyens raisonnables, sans toutefois suggérer que L'Offrant vous soutient ou soutient la façon dont vous avez utilisé son Oeuvre.

Pas de restrictions additionnelles — Vous n'êtes pas autorisé à appliquer des conditions légales ou des mesures techniques qui restreindraient légalement autrui à utiliser l'Oeuvre dans les conditions décrites par la licence.

Pour consulter une copie de cette licence, visitez : {dataset_metadata['license_url']}
"""

license_path = os.path.join(deposit_path, 'LICENSE.txt')
with open(license_path, 'w', encoding='utf-8') as f:
    f.write(license_content)
print(f"- Fichier 'LICENSE.txt' créé dans '{deposit_path}'.")

## 6. (Optionnel) Archivage du Paquet de Dépôt en ZIP

Pour faciliter le transfert ou le dépôt, il peut être utile de créer une archive ZIP du paquet.

In [ ]:
def zip_deposit_package(package_path, archive_name_base):
    # L'archive sera créée dans le dossier PARENT du package_path (c.a.d. OUTPUT_DIR_BASE)
    # et aura le nom du dossier du paquet.
    archive_output_filename = os.path.join(os.path.dirname(package_path), archive_name_base)
    try:
        shutil.make_archive(archive_output_filename, 'zip', root_dir=os.path.dirname(package_path) , base_dir=os.path.basename(package_path))
        print(f"\nArchive ZIP '{archive_output_filename}.zip' créée avec succès.")
        return f"{archive_output_filename}.zip"
    except Exception as e:
        print(f"Erreur lors de la création de l'archive ZIP : {e}")
        return None

zip_file_path = zip_deposit_package(deposit_path, DEPOSIT_PACKAGE_NAME)
if zip_file_path:
    print(f"Le paquet de dépôt est prêt et archivé ici : {os.path.abspath(zip_file_path)}")

## 7. Simulation de la Démarche de Dépôt sur NAKALA

**NAKALA** (proposé par Huma-Num) est un entrepôt de données pour les Sciences Humaines et Sociales. Il permet de déposer des jeux de données, de leur assigner des métadonnées riches et un identifiant pérenne (DOI), assurant leur citabilité et leur réutilisabilité.

**Étapes générales pour un dépôt sur NAKALA (non exécutées par ce notebook) :**

1.  **Création d'un compte / Connexion :** Accéder à la plateforme NAKALA ([https://www.nakala.fr](https://www.nakala.fr)) et se connecter via son compte Huma-ID (ou en créer un).
2.  **Nouveau Dépôt :** Dans l'interface NAKALA, initier un nouveau dépôt de type "Donnée" (ou "Collection" si on dépose un ensemble de données liées).
3.  **Remplissage des Métadonnées :**
    * NAKALA propose un formulaire web pour saisir les métadonnées. Les informations préparées dans notre fichier `metadata_dublincore.xml` (ou `metadata_simple.json`) peuvent être utilisées pour remplir ce formulaire (Titre, Auteur, Description, Mots-clés, Licence, etc.).
    * NAKALA utilise principalement les schémas Dublin Core et DataCite. Il est important de bien renseigner les champs obligatoires et recommandés pour assurer une bonne découvrabilité.
4.  **Téléversement des Fichiers (Upload) :**
    * Téléverser les fichiers du paquet de dépôt (`data/`, `images/`, `documentation/`, `LICENSE.txt`, `README_dataset.md`).
    * NAKALA permet généralement de téléverser les fichiers individuellement ou, pour des structures complexes, de téléverser une archive ZIP (comme celle que nous avons créée) puis de la "décompresser" sur le serveur. Il est aussi possible de référencer des fichiers déjà présents ailleurs via leur URL.
    * Il est possible d'associer des métadonnées spécifiques à chaque fichier téléversé, en plus des métadonnées globales du jeu de données.
5.  **Vérification et Publication :**
    * Vérifier toutes les informations et les fichiers.
    * Choisir le niveau de visibilité (public, privé, restreint).
    * Soumettre le dépôt. Une fois validé (parfois après une modération), NAKALA assigne un **DOI (Digital Object Identifier)** unique et pérenne au jeu de données.

**Conseils pour Sophie Bernard :**
* **Préparer les métadonnées avec soin :** Des métadonnées de qualité sont essentielles.
* **Consulter la documentation de NAKALA :** Huma-Num fournit une documentation détaillée et des guides pour les déposants.
* **Anticiper le Plan de Gestion de Données (PGD/DMP) :** De nombreux projets de recherche exigent un PGD. Penser au dépôt dès le début du projet facilite sa réalisation.
* **Contacter les services d'appui :** Les ingénieurs d'Huma-Num ou les services d'appui à la recherche de son institution peuvent l'accompagner.

## 8. Conclusion et Pistes Futures (pour Sophie Bernard)

Ce notebook a guidé Sophie Bernard à travers les étapes de préparation de son jeu de données archéologiques pour un dépôt FAIR dans un entrepôt comme NAKALA. Le "paquet de dépôt" créé contient :
* Les données organisées (CSV, images, rapport).
* Un fichier de métadonnées descriptives standardisé (`metadata_dublincore.xml`).
* Une documentation claire (`README_dataset.md`).
* Une licence de réutilisation explicite (`LICENSE.txt`).

**Prochaines étapes :**
1.  **Vérifier et compléter** les métadonnées et le `README_dataset.md` avec toutes les informations spécifiques à son travail réel.
2.  **Effectuer le dépôt effectif** sur NAKALA (ou un autre entrepôt pertinent pour sa discipline et son institution).
3.  **Valoriser son jeu de données :** Une fois le DOI obtenu, le citer dans ses publications, envisager la rédaction d'un "Data Paper" pour décrire le jeu de données.

Cet accompagnement a pour but de démystifier le processus de dépôt et d'encourager les bonnes pratiques en matière de gestion et de partage des données de la recherche.